In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, TimeDistributed, LSTM, Dense, Flatten
from keras.utils import to_categorical

# Function to load video dataset
def load_video_dataset(dataset_path, num_frames):
    X = []
    y = []
    
    # Iterate through each subdirectory (class label) in the dataset
    for class_label in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_label)
        
        # Check if the subdirectory is a directory
        if not os.path.isdir(class_path):
            continue
        
        # Assign a numerical label to the class (0 for the first class, 1 for the second class)
        if class_label == "Fight":  # Replace "class1" with the actual name of your first class folder
            label = 0
        elif class_label == "NonFight":  # Replace "class2" with the actual name of your second class folder
            label = 1
        else:
            continue  # Skip if the subdirectory is neither "class1" nor "class2"
        
        # Iterate through each video file in the subdirectory
        for video_file in os.listdir(class_path):
            video_path = os.path.join(class_path, video_file)
            print("Reading video:", video_path)  # Add this line to check video path
            
            # Read the video using OpenCV
            cap = cv2.VideoCapture(video_path)
            frames = []
            for _ in range(num_frames):
                ret, frame = cap.read()
                if not ret:
                    print("End of video reached.")
                    break
                frame = cv2.resize(frame, (224, 224))  # Resize frame if necessary
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
                frames.append(frame)
            
            cap.release()
            
            # Pad frames if necessary to ensure consistency in the number of frames
            while len(frames) < num_frames:
                frames.append(np.zeros_like(frames[0]))
            
            # Convert frames to numpy array and append to X
            X.append(np.array(frames))
            y.append(label)
    
    # Convert X and y to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    return X, y

# Load dataset
dataset_path = "D:/Git-Uploads/Main Project/Main-Project/violence_data/val"
num_frames = 150  # Number of frames per video
X, y = load_video_dataset(dataset_path, num_frames)

# Convert class labels to one-hot encoding
y_one_hot = to_categorical(y)

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# Define CNN-LSTM model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(num_frames, 224, 224, 1)))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(64, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Conv2D(128, (3, 3), activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(256, return_sequences=False))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Fight / No Fight 2 classes

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, batch_size=8, epochs=5, validation_data=(X_test, y_test))

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\0Ow4cotKOuw_0.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\0Ow4cotKOuw_1.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\0Ow4cotKOuw_2.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\0Ow4cotKOuw_3.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\0Ow4cotKOuw_4.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\1Kbw1bUw_0.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\1Kbw1bUw_1.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\1Kbw1bUw_2.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\1Kbw1bUw_3.avi
Reading video: D:/Git-Uploads/Main Project/Main-Project/violence_data/val\Fight\1MVS2QPWbHc_0.avi
Reading video: D:/Git-Uploads/Ma

ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/time_distributed_2/conv2d_1/Relu' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "d:\JUNK\Anaconda\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "d:\JUNK\Anaconda\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "d:\JUNK\Anaconda\Lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "d:\JUNK\Anaconda\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "d:\JUNK\Anaconda\Lib\asyncio\base_events.py", line 607, in run_forever
      self._run_once()
    File "d:\JUNK\Anaconda\Lib\asyncio\base_events.py", line 1922, in _run_once
      handle._run()
    File "d:\JUNK\Anaconda\Lib\asyncio\event